In [ ]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 9.3 MB/s 


In [ ]:
#Load packages
import pulp
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data21 = pd.read_csv('2021.csv')
data21 = data21['MTU (CET/CEST),"Day-ahead Price [EUR/MWh]","Currency","BZN|DE-LU"'].str.split(',', expand=True)
data21[['start_date','end_date']] = data21[0].str.split('-', expand=True)
del data21[0]
del data21['end_date']
data21.rename(columns = {1:'price', 2:'currency'}, inplace = True)
data21['price']=data21['price'].str.replace('"', '')
data21['currency']=data21['currency'].str.replace('"', '')
data21['price'] = data21['price'].replace(r'^\s*$', np.nan, regex=True)
data21['price'] = data21['price'].astype(float)
data21['start_date'] = pd.to_datetime(data21['start_date'])
data21['start_time']=data21['start_date'].dt.strftime("%H:%M")
data21['start_date']=data21['start_date'].dt.date

In [ ]:
data21

,price,currency,start_date,start_time
0,50.87,EUR,2021-01-01,00:00
1,48.19,EUR,2021-01-01,01:00
2,44.68,EUR,2021-01-01,02:00
3,42.92,EUR,2021-01-01,03:00
4,40.39,EUR,2021-01-01,04:00
...,...,...,...,...
8756,32.49,EUR,2021-12-31,19:00
8757,0.18,EUR,2021-12-31,20:00
8758,0.08,EUR,2021-12-31,21:00
8759,5.10,EUR,2021-12-31,22:00


In [ ]:
def select_timestamp(time):
    hours = int(time.split(':')[0])
    return True if hours <= 8 or hours >= 18 else False

In [ ]:
data21['selection'] = data21['start_time'].apply(select_timestamp)

In [ ]:
data21 = data21[data21['selection']]

In [ ]:
def rename_timestamp(time):
    hours = int(time.split(':')[0])
    if hours >= 18:
        hours -= 9
    return f"{hours}:00"

In [ ]:
data21['start_time_new'] = data21['start_time'].apply(rename_timestamp)

In [ ]:
data21

,price,currency,start_date,start_time,selection,start_time_new
0,50.87,EUR,2021-01-01,00:00,True,0:00
1,48.19,EUR,2021-01-01,01:00,True,1:00
2,44.68,EUR,2021-01-01,02:00,True,2:00
3,42.92,EUR,2021-01-01,03:00,True,3:00
4,40.39,EUR,2021-01-01,04:00,True,4:00
...,...,...,...,...,...,...
8756,32.49,EUR,2021-12-31,19:00,True,10:00
8757,0.18,EUR,2021-12-31,20:00,True,11:00
8758,0.08,EUR,2021-12-31,21:00,True,12:00
8759,5.10,EUR,2021-12-31,22:00,True,13:00


In [ ]:
data21['Time Stamp'] = data21['start_date'].astype(str) + " " + data21['start_time_new'].astype(str)
data21 = data21.set_index(['Time Stamp'])
data21

,price,currency,start_date,start_time,selection,start_time_new
Time Stamp,,,,,,
2021-01-01 0:00,50.87,EUR,2021-01-01,00:00,True,0:00
2021-01-01 1:00,48.19,EUR,2021-01-01,01:00,True,1:00
2021-01-01 2:00,44.68,EUR,2021-01-01,02:00,True,2:00
2021-01-01 3:00,42.92,EUR,2021-01-01,03:00,True,3:00
2021-01-01 4:00,40.39,EUR,2021-01-01,04:00,True,4:00
...,...,...,...,...,...,...
2021-12-31 10:00,32.49,EUR,2021-12-31,19:00,True,10:00
2021-12-31 11:00,0.18,EUR,2021-12-31,20:00,True,11:00
2021-12-31 12:00,0.08,EUR,2021-12-31,21:00,True,12:00


In [ ]:
data21.index = pd.to_datetime(data21.index, format='%Y-%m-%d %H:%M')
data21

,price,currency,start_date,start_time,selection,start_time_new
Time Stamp,,,,,,
2021-01-01 00:00:00,50.87,EUR,2021-01-01,00:00,True,0:00
2021-01-01 01:00:00,48.19,EUR,2021-01-01,01:00,True,1:00
2021-01-01 02:00:00,44.68,EUR,2021-01-01,02:00,True,2:00
2021-01-01 03:00:00,42.92,EUR,2021-01-01,03:00,True,3:00
2021-01-01 04:00:00,40.39,EUR,2021-01-01,04:00,True,4:00
...,...,...,...,...,...,...
2021-12-31 10:00:00,32.49,EUR,2021-12-31,19:00,True,10:00
2021-12-31 11:00:00,0.18,EUR,2021-12-31,20:00,True,11:00
2021-12-31 12:00:00,0.08,EUR,2021-12-31,21:00,True,12:00


In [ ]:
data21 = data21.loc[:,['price']]

In [ ]:
data21

,price
Time Stamp,
2021-01-01 00:00:00,50.87
2021-01-01 01:00:00,48.19
2021-01-01 02:00:00,44.68
2021-01-01 03:00:00,42.92
2021-01-01 04:00:00,40.39
...,...
2021-12-31 10:00:00,32.49
2021-12-31 11:00:00,0.18
2021-12-31 12:00:00,0.08


In [ ]:
data21 = data21[~data21.index.duplicated(keep='first')]

In [ ]:
data21['price'].fillna((data21['price'].mean()), inplace=True)

In [ ]:
data21

,price
Time Stamp,
2021-01-01 00:00:00,50.87
2021-01-01 01:00:00,48.19
2021-01-01 02:00:00,44.68
2021-01-01 03:00:00,42.92
2021-01-01 04:00:00,40.39
...,...
2021-12-31 10:00:00,32.49
2021-12-31 11:00:00,0.18
2021-12-31 12:00:00,0.08


In [ ]:
data21.iloc[:15]

,price
Time Stamp,
2021-01-01 00:00:00,50.87
2021-01-01 01:00:00,48.19
2021-01-01 02:00:00,44.68
2021-01-01 03:00:00,42.92
2021-01-01 04:00:00,40.39
2021-01-01 05:00:00,40.20
2021-01-01 06:00:00,39.63
2021-01-01 07:00:00,40.09
2021-01-01 08:00:00,41.27


In [ ]:
class Battery():
    def __init__(self,
                 time_horizon,
                 max_discharge_power_capacity,
                 max_charge_power_capacity):
        #Set up decision variables for optimization.
        #These are the hourly charge and discharge flows for
        #the optimization horizon, with their limitations.
        self.time_horizon = time_horizon

        self.charge = \
        pulp.LpVariable.dicts(
            "charging_power",
            ('c_t_' + str(i) for i in range(0,time_horizon)),
            lowBound=0, upBound=max_charge_power_capacity,
            cat='Continuous')

        self.discharge = \
        pulp.LpVariable.dicts(
            "discharging_power",
            ('d_t_' + str(i) for i in range(0,time_horizon)),
            lowBound=0, upBound=max_discharge_power_capacity,
            cat='Continuous')

    def set_objective(self, prices):
        #Create a model and objective function.
        #This uses price data, which must have one price
        #for each point in the time horizon.
        try:
            assert len(prices) == self.time_horizon
        except:
            print('Error: need one price for each hour in time horizon')

        #Instantiate linear programming model to maximize the objective
        self.model = pulp.LpProblem("Energy arbitrage", pulp.LpMaximize)

        #Objective is profit
        #This formula gives the daily profit from charging/discharging
        #activities. Charging is a cost, discharging is a revenue
        self.model += \
        pulp.LpAffineExpression(
            [(self.charge['c_t_' + str(i)],
              -1*prices[i]) for i in range(0,self.time_horizon)]) +\
        pulp.LpAffineExpression(
            [(self.discharge['d_t_' + str(i)],
              prices[i]) for i in range(0,self.time_horizon)])

    def add_storage_constraints(self,
                                efficiency,
                                min_capacity,
                                discharge_energy_capacity,
                                initial_level):
        #Storage level constraint 1
        #This says the battery cannot have less than zero energy, at
        #any hour in the horizon
        #Note this is a place where round-trip efficiency is factored in.
        #The energy available for discharge is the round-trip efficiency
        #times the energy that was charged.
        for hour_of_sim in range(1,self.time_horizon+1):
            self.model += \
            initial_level \
            + pulp.LpAffineExpression(
                [(self.charge['c_t_' + str(i)], efficiency)
                 for i in range(0,hour_of_sim)]) \
            - pulp.lpSum(
                self.discharge[index]
                for index in('d_t_' + str(i)
                             for i in range(0,hour_of_sim)))\
            >= min_capacity

        #Storage level constraint 2
        #Similar to 1
        #This says the battery cannot have more than the
        #discharge energy capacity
        for hour_of_sim in range(1,self.time_horizon+1):
            self.model += \
            initial_level \
            + pulp.LpAffineExpression(
                [(self.charge['c_t_' + str(i)], efficiency)
                 for i in range(0,hour_of_sim)]) \
            - pulp.lpSum(
                self.discharge[index]
                for index in ('d_t_' + str(i)
                              for i in range(0,hour_of_sim)))\
            <= discharge_energy_capacity

        self.model += pulp.LpConstraint(self.charge['c_t_8'],sense=0,rhs=0)
        self.model += pulp.LpConstraint(self.discharge['d_t_8'],sense=0,rhs=10)


    def solve_model(self):
        #Solve the optimization problem
        self.model.solve()

        #Show a warning if an optimal solution was not found
        if pulp.LpStatus[self.model.status] != 'Optimal':
            print('Warning: ' + pulp.LpStatus[self.model.status])

    def collect_output(self):
        #Collect hourly charging and discharging rates within the
        #time horizon
        hourly_charges =\
            np.array(
                [self.charge[index].varValue for
                 index in ('c_t_' + str(i) for i in range(0,15))])
        hourly_discharges =\
            np.array(
                [self.discharge[index].varValue for
                 index in ('d_t_' + str(i) for i in range(0,15))])

        hourly_discharges_updated = []
        for i in range(0,15):
            if i != 8:
                hourly_discharges_updated.append(self.discharge['d_t_' + str(i)].varValue)
            else:
                hourly_discharges_updated.append(0)

        return hourly_charges, hourly_discharges, hourly_discharges_updated

In [ ]:
def simulate_battery(initial_level,
                     price_data,
                     max_discharge_power_capacity,
                     max_charge_power_capacity,
                     discharge_energy_capacity,
                     efficiency,
                     time_horizon,
                     start_day):
    #Track simulation time
    tic = time.time()

    #Initialize output variables
    all_hourly_charges = np.empty(0)
    all_hourly_discharges = np.empty(0)
    all_hourly_state_of_energy = np.empty(0)
    all_daily_discharge_throughput = np.empty(0)

    #Set up decision variables for optimization by
    #instantiating the Battery class
    battery = Battery(
        time_horizon=time_horizon,
        max_discharge_power_capacity=max_discharge_power_capacity,
        max_charge_power_capacity=max_charge_power_capacity)

    #############################################
    #Run the optimization for each day of the year.
    #############################################

    #There are 365 24-hour periods (noon to noon) in the simulation,
    #contained within 366 days
    for day_count in range(365):
        #print('Trying day {}'.format(day_count))

        #############################################
        ### Select data and simulate daily operation
        #############################################

        #Set up the 36 hour optimization horizon for this day by
        #adding to the first day/time of the simulation
        start_time = start_day \
        + pd.Timedelta(day_count, unit='days')
        end_time = start_time + pd.Timedelta(time_horizon-1, unit='hours')
        #print(start_time, end_time)

        #Retrieve the price data that will be used to calculate the
        #objective
        prices = \
        price_data[start_time:end_time]['price'].values

        #Create model and objective
        battery.set_objective(prices)

        #Set storage constraints
        battery.add_storage_constraints(
            efficiency=efficiency,
            min_capacity=0,
            discharge_energy_capacity=discharge_energy_capacity,
            initial_level=initial_level)


        #Solve the optimization problem and collect output
        battery.solve_model()
        hourly_charges, hourly_discharges, hourly_discharges_updated = battery.collect_output()

        #############################################
        ### Manipulate daily output for data analysis
        #############################################

        #Collect daily discharge throughput
        daily_discharge_throughput = sum(hourly_discharges)
        #Calculate net hourly power flow (kW), needed for state of energy.
        #Charging needs to factor in efficiency, as not all charged power
        #is available for discharge.
        net_hourly_activity = (hourly_charges*efficiency) \
        - hourly_discharges
        #Cumulative changes in energy over time (kWh) from some baseline
        cumulative_hourly_activity = np.cumsum(net_hourly_activity)
        #Add the baseline for hourly state of energy during the next
        #time step (t2)
        state_of_energy_from_t2 = initial_level \
        + cumulative_hourly_activity

        #Append output
        all_hourly_charges = np.append(all_hourly_charges, hourly_charges)
        all_hourly_discharges = np.append(
            all_hourly_discharges, hourly_discharges_updated)
        all_hourly_state_of_energy = \
        np.append(all_hourly_state_of_energy, state_of_energy_from_t2)
        all_daily_discharge_throughput = \
        np.append(
            all_daily_discharge_throughput, daily_discharge_throughput)

        #############################################
        ### Set up the next day
        #############################################

        #Initial level for next period is the end point of current period
        initial_level = state_of_energy_from_t2[-1]



    toc = time.time()

    print('Total simulation time: ' + str(toc-tic) + ' seconds')

    return all_hourly_charges, all_hourly_discharges, \
        all_hourly_state_of_energy,\
        all_daily_discharge_throughput

In [ ]:
max_discharge_power_capacity = 10 #(kW)
max_charge_power_capacity = 10 #(kW)
discharge_energy_capacity = 50 #(kWh)
efficiency = 1.0 #unitless
initial_level = 50

In [ ]:
all_hourly_charges, all_hourly_discharges, all_hourly_state_of_energy,\
all_daily_discharge_throughput = \
simulate_battery(initial_level=initial_level,
                 price_data=data21,
                 max_discharge_power_capacity
                     =max_discharge_power_capacity,
                 max_charge_power_capacity
                     =max_charge_power_capacity,
                 discharge_energy_capacity=discharge_energy_capacity,
                 efficiency=efficiency,
                 time_horizon=15,
                 start_day=pd.Timestamp(
                     year=2021, month=1, day=1, hour=0))

Total simulation time: 10.44048023223877 seconds


In [ ]:
data21_final = data21.copy()

In [ ]:
#These indicate flows during the hour of the datetime index
data21_final['Charging power (kW)'] = all_hourly_charges
data21_final['Discharging power (kW)'] = all_hourly_discharges
data21_final['Power output (kW)'] = \
    all_hourly_discharges - all_hourly_charges
#This is the state of power at the beginning of the hour of the datetime index
data21_final['State of Energy (kWh)'] = \
    np.append(initial_level, all_hourly_state_of_energy[0:-1])

In [ ]:
data21_final['Revenue generation ($)'] = \
data21_final['Discharging power (kW)'] \
* data21_final['price'] / 1000

In [ ]:
data21_final['Charging cost ($)'] = \
data21_final['Charging power (kW)'] \
* data21_final['price'] / 1000

In [ ]:
data21_final['Profit ($)'] = data21_final['Revenue generation ($)'] \
- data21_final['Charging cost ($)']

In [ ]:
data21_final['Revenue generation ($)'].sum()

2770.4631441907195

In [ ]:
data21_final['Charging cost ($)'].sum()

2361.0905000000002

In [ ]:
data21_final['Profit ($)'].sum()

409.3726441907198

In [ ]:
data21_final[:50]

,price,Charging power (kW),Discharging power (kW),Power output (kW),State of Energy (kWh),Revenue generation ($),Charging cost ($),Profit ($)
Time Stamp,,,,,,,,
2021-01-01 00:00:00,50.87,0.0,10.0,10.0,50.0,0.5087,0.0000,0.5087
2021-01-01 01:00:00,48.19,0.0,10.0,10.0,40.0,0.4819,0.0000,0.4819
2021-01-01 02:00:00,44.68,0.0,10.0,10.0,30.0,0.4468,0.0000,0.4468
2021-01-01 03:00:00,42.92,0.0,10.0,10.0,20.0,0.4292,0.0000,0.4292
2021-01-01 04:00:00,40.39,10.0,0.0,-10.0,10.0,0.0000,0.4039,-0.4039
2021-01-01 05:00:00,40.20,10.0,0.0,-10.0,20.0,0.0000,0.4020,-0.4020
2021-01-01 06:00:00,39.63,10.0,0.0,-10.0,30.0,0.0000,0.3963,-0.3963
2021-01-01 07:00:00,40.09,10.0,0.0,-10.0,40.0,0.0000,0.4009,-0.4009
2021-01-01 08:00:00,41.27,0.0,0.0,0.0,50.0,0.0000,0.0000,0.0000
